In [10]:
import numpy as np
import time

class gaussSolve:
    '''Class for solving linear equation with gaussian elimination'''
        
    def __init__(self,A0,b0,):
        self.A0=A0
        self.b0=b0
        self.N=len(b0)
    
        Aandb=self.forward_elim(A0,b0)
        self.Aech=Aandb[0]
        self.bEch=Aandb[1]
        
        self.x=self.back_substitute(self.Aech,self.bEch)

        self.det=self.determinant(A0)
        self.inv=self.inverse(A0)        
        
    def __repr__(self):
        return 'A original:\n'+str(self.A0)+'\n b orignial: '+str(self.b0) \
                +'\n A echelon: \n'+str(self.Aech)+'\n b echelon: '+str(self.bEch) \
                + '\n \n Determinant: ' + str(self.det) +'\n Inverse: \n'+str(self.inv) \
                +'\n \n x: '+str(self.x)
   
    def forward_elim(self,A0,b0):
        '''Perform forward elimination steps'''
        
        A=np.copy(A0)
        b=np.copy(b0)
        
        nPivots=0
        for column in range(self.N):
        
            A,pivoted=self.pivot(A,b,column)
            if pivoted: nPivots+=1
            
            for row in range(column+1,self.N):              
                try:
                    multFac=(A[row,column]/A[column,column])
                except:
                    print("Issue with forward elimination!")
                A[row,:]-=multFac*A[column,:]
                b[row]-=multFac*b[column]

        return A,b,nPivots
            
    def back_substitute(self,A,b):
        '''Perform back substitution on row echelon matrix'''
        
        x=np.zeros(self.N)
        x[-1]=b[-1]/A[self.N-1,self.N-1]
        
        for row in range(self.N-2,-1,-1):
            for column in range(self.N-1,row,-1):
                x[row]-=x[column]*A[row,column]/A[row,row]
            x[row]+=b[row]/A[row,row]
            
        return x
            
    def pivot(self,A,b,column):
        '''Puts row with largest scaled value in column at row'''
        
        # Make sure we are only looking below row=column
        pivotRow=np.argmax((A[:,column]/np.max(np.abs(A), 1))[column:self.N])+column
        A[[column,pivotRow]]=A[[pivotRow,column]]
        b[[column,pivotRow]]=b[[pivotRow,column]]
        
        pivoted=True if column != pivotRow else False
        
        return A,pivoted

    def determinant(self,A,isEchelon=False,nPivots=-1):
        '''Calculate the determinant Gauss elimination'''
        
        if isEchelon: 
            if nPivots > 0:
                return (-1.0)**nPivots*np.prod(np.diagonal(A))
            else:
                print('Must supply nPivots for determinant of Echelon matrix')
                raise
        
        bDummy=np.zeros(self.N)
        
        Aech,bDummy,nPivots=self.forward_elim(A,bDummy)
        
        return (-1.0)**nPivots*np.prod(np.diagonal(Aech))
    
    def inverse(self,A0):
        '''Calculate the inverse via Gauss elimination'''
        
        invA=np.identity(self.N)
        
        for column in range(self.N):
            A,e,nPivots=self.forward_elim(A0,invA[:,column])
            x=self.back_substitute(A,e)
            
            invA[:,column]=x
        
        return invA
            

In [11]:
# Some examples:

A=np.array([[1,1,1],[-1,2,0],[2,0,1]],float)
b=np.array([6.0,3.0,5.0])

#eps=1e-10
#A=np.array([[eps,1,1],[1,1,0],[1,0,1]],float)
#b=np.array([5.0,3.0,4.0])

#nMatrix=3
#A=np.random.rand(nMatrix,nMatrix)
#b=np.random.rand(nMatrix)

gaussTest=gaussSolve(A,b)

print(gaussTest,'\n')
print('numpy determinant: ',np.linalg.det(A),'\n')
print('numpy inverse: \n',np.linalg.inv(A),'\n')
print('numpy solve x: ',np.linalg.solve(A,b),'\n')


A original:
[[ 1.  1.  1.]
 [-1.  2.  0.]
 [ 2.  0.  1.]]
 b orignial: [6. 3. 5.]
 A echelon: 
[[ 1.          1.          1.        ]
 [ 0.          3.          1.        ]
 [ 0.          0.         -0.33333333]]
 b echelon: [ 6.  9. -1.]
 
 Determinant: -1.0
 Inverse: 
[[-2.  1.  2.]
 [-1.  1.  1.]
 [ 4. -2. -3.]]
 
 x: [1. 2. 3.] 

numpy determinant:  -1.0 

numpy inverse: 
 [[-2.  1.  2.]
 [-1.  1.  1.]
 [ 4. -2. -3.]] 

numpy solve x:  [1. 2. 3.] 



In [12]:
# Example of a singular matrix:

A = np.array([ [ 1, 2, 3], 
               [ 4, 5, 6], 
               [ 7, 8, 9] ], dtype=np.float64)
b = np.array([5, -1, 2], dtype=np.float64)

gaussSing=gaussSolve(A,b)

print(gaussSing)

A original:
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
 b orignial: [ 5. -1.  2.]
 A echelon: 
[[7.         8.         9.        ]
 [0.         0.42857143 0.85714286]
 [0.         0.         0.        ]]
 b echelon: [ 2.         -2.14285714  9.        ]
 
 Determinant: -0.0
 Inverse: 
[[ nan  nan  nan]
 [-inf  inf -inf]
 [ inf -inf  inf]]
 
 x: [ nan -inf  inf]


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in true_divide
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in double_scalars
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in double_scalars
